In [2]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from python_module.pricing_model import SABRModel
from python_module.tools import maximize_with_bounds

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Pricing parameters
F     = 5000
alpha = 0.25
beta  = 1.0
rho   = -0.
nu    = 1.5
r     = 0
bd    = 20
T     = bd / 250

# Scenario parameters
slide_to_compute = [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]
#slide_to_compute = [-0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05]


strike_list = []
for delta in [-0.01, -0.1, -0.2, -0.3, -0.4, 0.5, 0.4, 0.3, 0.2, 0.1, 0.01]:
    option_type = 'call' if delta > 0 else 'put'
    K = SABRModel.solve_delta_strike(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type=option_type, target_delta=delta)
    K = int(K)
    strike_list.append(K)

#if min(strike_list) > F*0.8:
#    strike_list.append(int(F*0.8))

#if F*1.2 > max(strike_list):
#    strike_list.append(int(F*1.2))

strike_list = sorted(strike_list)

market_data_list = list()
for K in strike_list:

    option_type = 'call' if K >= F else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)

    market_data_list.append({
        'symbol': f"{bd}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': bd, 
        'F':F, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [4]:
market_data_df

,option_type,time_to_maturity,F,r,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,-0.30,-0.10,-0.05,-0.04,-0.03,-0.02,-0.01,0.01,0.02,0.03,0.04,0.05,0.10,0.30
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20T_4339K_put,put,20,5000,0,4339,0.08,0.28,5.36,-0.03,0.00,1.05,-0.74,-0.47,"1,209.63",784.11,40.74,5.14,2.62,1.02,0.15,-0.15,0.49,1.25,2.22,3.34,4.59,11.88,44.88
20T_4564K_put,put,20,5000,0,4564,0.08,0.27,19.50,-0.11,0.00,2.57,-1.71,-0.81,"1,430.37",886.93,92.30,15.71,8.80,4.08,1.23,-0.04,1.12,3.10,5.78,9.01,12.68,34.92,138.30
20T_4716K_put,put,20,5000,0,4716,0.08,0.26,42.79,-0.20,0.00,3.98,-2.57,-0.83,979.46,870.73,124.81,25.99,15.42,7.76,2.82,0.32,1.56,4.71,9.21,14.82,21.33,62.38,259.86
20T_4828K_put,put,20,5000,0,4828,0.08,0.26,71.98,-0.30,0.00,4.93,-3.15,-0.61,449.51,803.97,136.61,32.49,20.03,10.63,4.25,0.77,1.71,5.63,11.50,19.03,27.97,86.67,380.26
20T_4926K_put,put,20,5000,0,4926,0.08,0.25,108.31,-0.40,0.00,5.48,-3.48,-0.26,90.01,711.96,135.36,35.59,22.62,12.50,5.35,1.20,1.64,5.95,12.69,21.61,32.45,106.92,497.67
20T_5000K_call,call,20,5000,0,5000,0.08,0.25,143.13,0.51,0.00,5.64,-3.58,0.06,-2.86,628.36,127.71,35.84,23.26,13.21,5.91,1.48,1.46,5.79,12.84,22.40,34.26,119.10,585.71
20T_5104K_call,call,20,5000,0,5104,0.08,0.25,98.88,0.40,0.00,5.47,-3.48,0.49,173.17,503.00,109.99,33.33,22.19,13.04,6.14,1.73,1.09,5.06,11.92,21.59,33.95,128.61,695.79
20T_5206K_call,call,20,5000,0,5206,0.08,0.26,66.79,0.30,0.00,4.93,-3.16,0.81,594.39,385.02,88.85,28.60,19.44,11.75,5.77,1.78,0.67,3.97,10.04,18.93,30.66,127.65,776.27
20T_5324K_call,call,20,5000,0,5324,0.08,0.26,41.23,0.21,0.00,4.04,-2.62,0.98,"1,135.53",268.51,64.88,22.06,15.30,9.50,4.86,1.61,0.26,2.65,7.37,14.62,24.53,114.95,826.57


In [5]:
slide_df = market_data_df.loc[:, [-0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05]] 
theta = market_data_df['theta']
theta.name = 0
scenarios = pd.concat([slide_df, theta], axis=1)
scenarios = scenarios.transpose().sort_index().transpose()

In [6]:
scenarios

,-0.05,-0.04,-0.03,-0.02,-0.01,0.00,0.01,0.02,0.03,0.04,0.05
symbol,,,,,,,,,,,
20T_4339K_put,5.14,2.62,1.02,0.15,-0.15,-0.74,0.49,1.25,2.22,3.34,4.59
20T_4564K_put,15.71,8.80,4.08,1.23,-0.04,-1.71,1.12,3.10,5.78,9.01,12.68
20T_4716K_put,25.99,15.42,7.76,2.82,0.32,-2.57,1.56,4.71,9.21,14.82,21.33
20T_4828K_put,32.49,20.03,10.63,4.25,0.77,-3.15,1.71,5.63,11.50,19.03,27.97
20T_4926K_put,35.59,22.62,12.50,5.35,1.20,-3.48,1.64,5.95,12.69,21.61,32.45
20T_5000K_call,35.84,23.26,13.21,5.91,1.48,-3.58,1.46,5.79,12.84,22.40,34.26
20T_5104K_call,33.33,22.19,13.04,6.14,1.73,-3.48,1.09,5.06,11.92,21.59,33.95
20T_5206K_call,28.60,19.44,11.75,5.77,1.78,-3.16,0.67,3.97,10.04,18.93,30.66
20T_5324K_call,22.06,15.30,9.50,4.86,1.61,-2.62,0.26,2.65,7.37,14.62,24.53


In [7]:
target_scenario = -0.05
df = scenarios.drop(target_scenario, axis=1)
s = scenarios[target_scenario]

In [8]:
# Constrain weights between -0.5 and 0.8
w_opt, res = maximize_with_bounds(df=df, s=s, w_min=-10000, w_max=10000)

print("Optimal weights:\n", w_opt)
print("Maximized objective (w^T s):", float(w_opt.dot(s)))

Optimal weights:
 symbol
20T_4339K_put    -10,000.00
20T_4564K_put    -10,000.00
20T_4716K_put    -10,000.00
20T_4828K_put      7,829.54
20T_4926K_put     10,000.00
20T_5000K_call    10,000.00
20T_5104K_call    10,000.00
20T_5206K_call    -9,312.29
20T_5324K_call   -10,000.00
20T_5516K_call   -10,000.00
20T_5815K_call   -10,000.00
Name: weights, dtype: float64
Maximized objective (w^T s): 175775.99706054755


In [9]:
market_data_df.loc[:, [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]].multiply(w_opt, axis=0).sum(axis=0)

-0.30    -8,767,113.32
-0.10       281,511.61
-0.05       175,776.00
-0.04       112,895.34
-0.03        58,695.76
-0.02        19,793.98
-0.01             0.00
0.01         17,358.70
0.02         46,827.37
0.03         80,899.64
0.04        110,526.93
0.05        125,893.69
0.10       -325,206.25
0.30    -13,771,373.84
dtype: float64

In [14]:
market_data_df.loc[:, [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 'theta', 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]].loc['20T_5000K_call']*10_000

-0.30    6,283,630.31
-0.10    1,277,115.38
-0.05      358,406.13
-0.04      232,579.61
-0.03      132,131.11
-0.02       59,064.61
-0.01       14,787.91
 theta     -35,767.74
0.01        14,641.39
0.02        57,914.54
0.03       128,368.92
0.04       224,034.80
0.05       342,583.56
0.10     1,190,964.49
0.30     5,857,122.45
Name: 20T_5000K_call, dtype: float64

In [11]:
market_data_df['vega']* w_opt

symbol
20T_4339K_put    -10,537.45
20T_4564K_put    -25,747.21
20T_4716K_put    -39,795.96
20T_4828K_put     38,579.96
20T_4926K_put     54,772.12
20T_5000K_call    56,382.64
20T_5104K_call    54,681.28
20T_5206K_call   -45,872.46
20T_5324K_call   -40,355.02
20T_5516K_call   -25,451.95
20T_5815K_call   -10,215.32
dtype: float64